In [ ]:
# !pip install pygwalker

In [ ]:
import pygwalker as pyg

In [ ]:
# 경로설정
import sys
path = "C:\github/aide-analysis/"
path2 = 'C:\github/aide-analysis/notebook/temp_ljs/Develop/지주리포트Lite/실거래 이상값 제거 연구 및 추론/모듈 및 압축파일 보관 폴더'
sys.path.append(path)
sys.path.append(path2)

import os
import sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd())))
# 한글폰트
import os
import matplotlib.pyplot as plt
if os.name =='posix':
    plt.rc('font',family = 'ApplGothic')
else:
    plt.rc('font',family = 'Malgun Gothic')
    
# 경고출력문 제거
import warnings
warnings.filterwarnings("ignore")

# 모듈 import
import pandas as pd
import src
from src.openapi import KakaoLocalAPI as kakao
from openpyxl import load_workbook
from src.utils import insert_dataframe_to_excel_sheet
from src.sql import DatabaseHandler
from src.temp import *
from datetime import datetime
import datetime
from dateutil.relativedelta import relativedelta
from src.bigquery import BigqueryHandler
import seaborn as sns
import numpy as np
import statistics
from tqdm import tqdm
import config
from core.config import Config
import json
from datetime import datetime
import geopandas as gpd
from tqdm import tqdm
import folium
from src.bigquery import bigquery_client, bigquery
import geopandas as gpd
from shapely import wkt
from openpyxl import load_workbook
from src.utils import insert_dataframe_to_excel_sheet
from openpyxl.utils.dataframe import dataframe_to_rows
from difflib import SequenceMatcher
import openpyxl
import math
from 실거래전처리 import TradeProcessing

config = Config()
BH = BigqueryHandler()


# DataFrame 디스플레이 설정
pd.set_option('display.max_columns', 250)
pd.set_option('display.max_rows', 250)
pd.set_option('display.width', 100)

# 지난달(23년01월)대비 이번달(23년02월) 매매가격이 떨어진 단지 찾기

- 타입별로 특수성이 있을 수 있기때문에 areaSixGroupNo 대신 pyeongNo를 기준으로 동 타입기준으로 가격비교 하기 -> 이렇게 했을때 문제가 없을지도 검증이 필요하다

In [ ]:
# sql = f"""

# SELECT sigunguCode
# FROM `aidepartners.aide.code_sigungu`
# WHERE (isMulti = True AND guLevel = True AND siLevel = False) OR (isMulti = False AND guLevel = True AND siLevel = True)
# """
# sigunguList = BH.read_table(sql)
# sigunguList = list(sigunguList['sigunguCode'])

In [ ]:
sql = f"""
SELECT distinct sidoCode
FROM `aidepartners.aide.code_sigungu`
"""
sidoList = BH.read_table(sql)
sidoList = list(sidoList['sidoCode'])

In [ ]:
sql =f"""
SELECT T1.*
FROM(
    SELECT sidoCode,
           complexNo, 
           complexName, 
           address,
           areaSixGroupNo, 
           pyeongNo,
           useApproveYear, 
           totalHouseholdCount, 
           tradeType, 
           jeonyongArea,
           supplyArea,
           contractDate, 
           averagePyeong, 
           averagePrice, 
           isFloor, 
           isProcessing1, 
           isProcessing2, 
           isProcessing3, 
           interpolate, 
           yearMonth, 
           isReal, 
           finalProcessing, 
           predict,
           RANK() OVER(PARTITION BY complexNo, complexName, pyeongNo ORDER BY yearMonth DESC) AS rn -- 월 구분
    FROM `aidepartners.raw.aide_apartment_price_origin`
    WHERE -- sidoCode ='28'
          -- sidoCode ='11' 
          (yearMonth >='2023-01-01' OR yearMonth <='2023-03-01')
          AND isReal = True
    ORDER BY complexNo, areaSixGRoupNo
    ) AS T1
"""
dataframe = BH.read_table(sql)

# 전달대비 하락한 지역 추론을 위한 데이터프레임 copy 및 전처리
data = dataframe.copy()
dropIdx = data[data['sidoCode'].isna()].index
data = data.drop(index = dropIdx)

In [ ]:
# 거래가 두달 연속 있는 단지중 전달대비 하락한 단지
def findFallPrice(data):
    result = pd.DataFrame()
    for sido in sidoList:
        df = data[data['sidoCode']==sido].reset_index(drop = True)
        df_result = pd.DataFrame()
        df_pecent = pd.DataFrame({'per':[]})
        df_unique  = df[['complexNo','pyeongNo','jeonyongArea', 'supplyArea']].drop_duplicates().reset_index(drop =True)
        for i in range(0, len(df_unique)):
            cNo =df_unique['complexNo'].iloc[i]
            typeOption = df_unique['pyeongNo'].iloc[i]
            # 거래가 두달이상 연속으로 있는경우
            if len(df.loc[(df['complexNo']==cNo)&(df['pyeongNo']==typeOption), 'rn'].unique()) >1:
                # 최근거래과 과거거래 금액 차이 비교
                recentPrice = df.loc[(df['complexNo']==cNo)&(df['pyeongNo']==typeOption)&(df['rn']==1),'averagePyeong'].mean()
                pastPrice = df.loc[(df['complexNo']==cNo)&(df['pyeongNo']==typeOption)&(df['rn']==2),'averagePyeong'].mean()
                price = pastPrice - recentPrice
                per = recentPrice / pastPrice # 전달대비 현재 실거래 비율
                if (price >0)&(per<=0.85) : # 전달 거래보다 싸게거래된 경우
                    df_result = df_result.append(df[(df['complexNo']==cNo)&(df['pyeongNo']==typeOption)])
                    if i ==0:
                        df_result['per'] = ''
                    df_result.loc[(df_result['complexNo']==cNo)&(df_result['pyeongNo']==typeOption),'per'] = per

                    # 결과테이블 세팅
                    df_result = df_result[['complexNo','complexName','address','pyeongNo','jeonyongArea', 'supplyArea','per']].drop_duplicates(subset = ['complexNo','pyeongNo']).sort_values(by = 'per')
                    df_result['per'] =  round(df_result['per'], 2)

                    # 컬럼정리
                    df_report = df_result.rename(columns = {'complexName':'단지명',
                                                            'pyeongNo':'면적타입',
                                                            'address':'주소',
                                                            'jeonyongArea':'전용면적',
                                                            'supplyArea':'공급면적',
                                                            'per':'전달 대비 실거래 가격'}).reset_index(drop = True)
                    result = result.append(df_report)
    result = result.reset_index(drop = True).sort_values(by = '전달 대비 실거래 가격')
    result = result.drop_duplicates(subset = ['단지명','면적타입']).reset_index(drop = True)
    return result

In [ ]:
result = findFallPrice(data)

In [ ]:
df_info = result[['complexNo','면적타입']].rename(columns = {'면적타입':'pyeongNo'})
sql =f"""
SELECT T1.*
FROM(
    SELECT sidoCode,
           complexNo, 
           complexName, 
           address,
           areaSixGroupNo, 
           pyeongNo,
           useApproveYear, 
           totalHouseholdCount, 
           tradeType, 
           jeonyongArea,
           supplyArea,
           contractDate, 
           averagePyeong, 
           averagePrice, 
           isFloor, 
           isProcessing1, 
           isProcessing2, 
           isProcessing3, 
           interpolate, 
           yearMonth, 
           isReal, 
           finalProcessing, 
           predict
    FROM `aidepartners.raw.aide_apartment_price_origin`
    WHERE yearMonth >='2021-01-01' 
          AND isReal = True
          AND complexNo in {tuple(df_info['complexNo'].unique())}
    ORDER BY complexNo, areaSixGRoupNo
df_info = result[['complexNo','면적타입']].rename(columns = {'면적타입':'pyeongNo'})    ) AS T1
"""
df_new = BH.read_table(sql)

In [ ]:
# # 최근하락한 단지의 최근2년 실거래 변동 확인
# for i in range(0, len(df_info)):
    
#     cNo = df_info['complexNo'].iloc[i]
#     pyeong = df_info['pyeongNo'].iloc[i]
    
#     # 단지-타입 특정 
#     df_unique = df_new[(df_new['complexNo']==cNo)&(df_new['pyeongNo']==pyeong)]
    
#     # 차트 제목에 사용할 파라미터 저장
#     danjiName = df_unique['complexName'].unique()
#     jeonyong = df_unique['jeonyongArea'].unique()
    
#     # 시각화
#     ## 실제값 필터
#     df_real = df_unique[df_unique['isReal']==True].reset_index(drop = True)
#     plt.figure(figsize = (12, 5))
#     sns.scatterplot(df_real['yearMonth'], df_real['averagePyeong'])
#     plt.title(f"단지명 : {danjiName}, 전용면적 : {jeonyong}")
#     plt.show()

# 전세가율

최근주만 사용할 경우 시세가 없는단지가 있음 

In [ ]:
sql = f"""
SELECT CASE WHEN provider = 'kab' THEN max(baseYearMonthDay) END AS kab,
       CASE WHEN provider = 'kbstar' THEN max(baseYearMonthDay) END AS kbstar,
       CASE WHEN provider = 'neonet' THEN max(baseYearMonthDay) END AS neonet
        
FROM `aidepartners.aide.complex_market_price`
GROUP BY provider
"""
time = BH.read_table(sql)
time

In [ ]:
day1 = time['kab'].dropna().iloc[0]
day2 = time['kbstar'].dropna().iloc[0]
day3 = time['neonet'].dropna().iloc[0]

In [ ]:
day = time.iloc[0][0]

In [ ]:
sql = f"""
SELECT danji.sidoCode,
       danji.sigunguCode,
       market.complexNo,
       danji.complexName,
       danji.sigunguName,
       detailAddress,
       market.areaNo,
       area.jeonyongArea,
       area.supplyArea,
       market.dealAveragePrice,
       market.leaseAveragePrice,
       market.leaseAveragePrice/market.dealAveragePrice AS ratio
FROM `aidepartners.aide.complex_market_price` AS market
LEFT JOIN (SELECT sidoCode,sigunguCode, complexNo, complexName, sigunguName, detailAddress FROM `aidepartners.aide.complex_danji_information`) AS danji ON market.complexNo = danji.complexNo
LEFT JOIN (SELECT complexNo, pyeongNo, jeonyongArea, supplyArea FROM `aidepartners.aide.complex_area_information_prod`) AS area ON market.complexNo =area.complexNo AND market.areaNo = area.pyeongNo
WHERE market.baseYearMonthDay = "{day}"
"""
df_ratio = BH.read_table(sql)

In [ ]:
df_ratio[(df_ratio['sidoCode']=='11')&(-df_ratio['complexName'].str.contains('도시형'))&(-df_ratio['complexName'].str.contains('주상복합'))].sort_values(by = 'ratio',ascending = False).head(200)[['complexName','sigunguName','jeonyongArea','supplyArea','ratio']].reset_index(drop = True)

# 오피스텔 유사단지의 광역비교

In [ ]:
sql = f"""
SELECT danji.complexNo,
       danji.complexName,
       danji.sidoCode,
       danji.sigunguCode,
       danji.bdongCode,
       danji.sigunguName,
       danji.bdongName,
       danji.totalHouseholdCount,
       danji.useApproveYear,
       danji.batlRatio,
       danji.btlRatio,
       danji.parkingPossibleCount,
       danji.parkingCountByHousehold,
       danji.HeatMethodTypeName,
       danji.HeatFuelTypeName,
       area.supplyArea,
       area.jeonyongArea,
       area.roomCount,
       area.entranceType,
       area.areaSixGroupNo,
       price.tradeType,
       price.tradeYear,
       price.tradeMonth,
       price.contractDate,
       price.dealPrice,
       price.leasePrice,
       price.rentPrice,
       price.floor
FROM `aidepartners.aide.complex_danji_information` AS danji

LEFT JOIN (
           SELECT complexNo, 
                  pyeongNo,
                  supplyArea,
                  jeonyongArea,
                  roomCount,
                  entranceType,
                  areaSixGroupNo          
           FROM `aidepartners.aide.complex_area_information_prod`
           ) AS area 
           ON danji.complexNo = area.complexNo
           
LEFT JOIN (
           SELECT complexNo, 
                  areaNo,
                  tradeType,
                  tradeYear,
                  tradeMonth,
                  contractDate,
                  dealPrice,
                  leasePrice,
                  rentPrice,
                  floor,
           FROM `aidepartners.aide.complex_real_price_prod`
           WHERE deleteYn is null
           ) AS price 
           ON area.complexNo = price.complexNo 
              AND area.pyeongNo = price.areaNo
WHERE danji.realEstateTypeCode = 'OPST' AND danji.sidoCode = '41'



"""
df = BH.read_table(sql)

In [ ]:
# 
df[(df['sigunguCode']=='41117')&(df['bdongName']=='이의동')].sort_values(by =['complexNo','jeonyongArea'])

In [ ]:
df[(df['sigunguCode']=='11500')&(df['bdongName']=='마곡동')].reset_index(drop = True).sort_values(by = ['complexNo','tradeYear'],ascending = False)

# 아파트 지역별 시세 조사

- type : 59/ 84
    - 59 : A2
    - 84 : A3

- sedaeCount  
    - 300이상
    - 2000 이상

- useApproveYear 
    - new : ~ 5년
    - general : 6년 ~ 20년
    - old :20년 ~

In [ ]:
sql = f"""
SELECT CASE WHEN provider = 'kab' THEN max(baseYearMonthDay) END AS kab,
       CASE WHEN provider = 'kbstar' THEN max(baseYearMonthDay) END AS kbstar,
       CASE WHEN provider = 'neonet' THEN max(baseYearMonthDay) END AS neonet
        
FROM `aidepartners.aide.complex_market_price`
GROUP BY provider
"""
time = BH.read_table(sql)
time

In [ ]:
day1 = time['kab'].dropna().iloc[0]
day2 = time['kbstar'].dropna().iloc[0]
day3 = time['neonet'].dropna().iloc[0]

In [ ]:
sql = f"""
SELECT danji.sidoCode,
       danji.sigunguCode,
       market.complexNo,
       danji.complexName,
       danji.sigunguName,
       detailAddress,
       market.areaNo,
       area.jeonyongArea,
       area.supplyArea,
       area.supplyPyeong,
       area.jeonyongPyeong,
       market.dealAveragePrice * 10000 AS dealAveragePrice,
       market.leaseAveragePrice * 10000 AS leaseAveragePrice,
       (market.dealAveragePrice * 10000)/ area.supplyPyeong AS dealAveragePricePerSupplyPyeong,
       (market.dealAveragePrice * 10000)/ area.jeonyongPyeong AS dealAveragePricePerJeonyongPyeong,
       market.leaseAveragePrice/market.dealAveragePrice AS ratio
       
FROM `aidepartners.aide.complex_market_price` AS market
LEFT JOIN (SELECT sidoCode,sigunguCode, complexNo, complexName, sigunguName, detailAddress FROM `aidepartners.aide.complex_danji_information`) AS danji ON market.complexNo = danji.complexNo
LEFT JOIN (SELECT complexNo, pyeongNo, jeonyongArea, supplyArea,jeonyongPyeong, supplyPyeong FROM `aidepartners.aide.complex_area_information_prod`) AS area ON market.complexNo =area.complexNo AND market.areaNo = area.pyeongNo
WHERE market.baseYearMonthDay in ("{day1}","{day2}","{day3}")
"""
df =BH.read_table(sql)

In [ ]:
gdf = pyg.walk(df)

In [ ]:
df[df['sidoCode']=='11'].drop_duplicates(subset = 'complexNo').reset_index(drop = True)['dealAveragePricePerSupplyPyeong'].mean()

In [ ]:
df[(df['sidoCode']=='11')&(-df['complexName'].str.contains('도시형'))&(-df['complexName'].str.contains('주상복합'))].drop_duplicates(subset = 'complexNo').reset_index(drop = True).sort_values(by = 'ratio',ascending = False)

# 아파트 가격추이

In [ ]:
startYear = datetime.now()-relativedelta(years = 5)

In [ ]:
class apartTimePrice():
    def __init__(self, sido, sigungu, name):
        """검색하고자 하는 시도명, 시군구명, 단지명을 입력받는다."""
        self.sido = sido
        self.sigungu = sigungu
        self.name = name

        
        
    def extractCode(self):
        """지역 코드 추출"""        
        sql = f"""
        SELECT sidoCode,
               sigunguCode,
        FROM `aidepartners.aide.code_sigungu`
        WHERE sidoName LIKE '%{self.sido}%'
              AND sigunguName LIKE '%{self.sigungu}%'
              AND ((isMulti = True AND guLevel = True AND siLevel = False) OR (isMulti = False AND guLevel = True AND siLevel = True))
        """
        code_data = BH.read_table(sql)
        if len(code_data) >1:
            raise Exception('입력받은 시도,시군구의 코드가 중복이 있습니다. 확인해 주세요') 
            
        else:
            sidoCode = code_data['sidoCode'][0]
            sigunguCode = code_data['sigunguCode'][0]
            
        return sidoCode, sigunguCode
        
        
    def extractDanjiName(self,sidoCode, sigunguCode):
        """단지 특정 및 추출"""

        sql = f"""
        SELECT complexNo, 
               complexName,
               address
        FROM `aidepartners.aide.complex_danji_information`
        WHERE sidoCode = "{sidoCode}"
              AND sigunguCode = "{sigunguCode}"
              AND complexName LIKE '%{self.name}%'        
        """
        danji_data = BH.read_table(sql)
        if len(danji_data) > 1:
            print('중복되는 단지들이 있습니다. 단지를 선택해주세요')
            print('-'*130)
            print('단지목록 : ',list(danji_data['complexName'].unique()))  
            print('-'*130)
            print('주소목록 : ',list(danji_data['address'].unique()))
            print('-'*130)
            answerName = input('앞에서 출력된 단지들중 해당하는 단지를 정확하게 입력해 주세요 : ')
            answerAddress = input('앞에서 출력된 주소중 해당하는 단지의 주소를 정확하게 입력해 주세요 : ')
            cNo = danji_data.loc[(danji_data['complexName']==answerName)&(danji_data['address']==answerAddress),'complexNo'].reset_index(drop = True)[0]
       
        else:
            answerName = danji_data['complexName'].reset_index(drop = True)[0]
            cNo = danji_data.loc[danji_data['complexName']==answerName,'complexNo'][0]
            
            
        return cNo
    
    
    def extractTargetPrice(self):
        """기준단지의 시계열 가격 데이터 추출"""
        
        # 시도, 시군구 코드 추출
        sidoCode, sigunguCode = self.extractCode()
        
        # 단지명 추출
        cNo = self.extractDanjiName(sidoCode, sigunguCode)
        
    
        # 타겟 단지의 실거래 시계열데이터 프레임 추출
        """입력받은 단지에 해당하는 실거래 시계열데이터 반환함수"""
        sql = f"""
        SELECT *
        FROM `aidepartners.aide.aide_apartment_price_origin`
        WHERE complexNo = "{cNo}"
        """
        df_target = BH.read_table(sql)

        if len(df_target) == 0:
            print('단지를 찾지못했습니다.')
        else:
            bdongCode = df_target['bdongCode'].iloc[0]
            
        df_target['yearMonth'] =pd.to_datetime(df_target['yearMonth'])
        return df_target, bdongCode
    
    def recentPrice(self):
        df_target, bdongCode = self.extractTargetPrice()
        startyear = datetime.now()-relativedelta(years = 5)
        sql = f"""
        SELECT *
        FROM `aideprtners.aide.aide_apartment_price_origin`
        WHERE sidoCode LIKE '{bdongCode[:2]}%'
              AND isReal = True
              AND yearMonth >={startYear}
        """
        df_sido = BH.read_table(sql)
        
        pass
#         BH.read_table(sql)
    
    
        


In [ ]:
startyear = datetime.now()-relativedelta(years = 5)
sql = f"""
SELECT complexName, 
       bdongCode, 
       useApproveYear, 
       pyeongNo, 
       jeonyongPyeong, 
       supplyPyeong, 
       areaSixGroupNo, 
       averagePyeong, 
       averagePrice,
       contractDate,
       yearMonth
FROM `aidepartners.aide.aide_apartment_price_origin`
WHERE sidoCode LIKE '31%'
      AND (isReal = True OR interpolate  = True)
      AND yearMonth >="{pd.to_datetime(startYear).date()}"
"""
df_total = BH.read_table(sql )

In [ ]:
df_total['areaSixGroupNo'].unique()

In [ ]:
# sns.lineplot(df_target.loc[df_target['isReal']==True,'yearMonth'], df_target.loc[df_target['isReal']==True,'averagePyeong'], hue = df_target['areaSixGroupNo'])
# df_total[df_total['bdongCode']==bdongCode].groupby('yearMonth')['averagePyeong'].mean().reset_index().plot(color = 'red')
# df_total[df_total['bdongCode'].str.contains(bdongCode[:5])].groupby('yearMonth')['averagePyeong'].mean().reset_index().plot(color = 'blue')
# df_total[df_total['bdongCode'].str.contains(bdongCode[:2])].groupby('yearMonth')['averagePyeong'].mean().reset_index().plot(color = 'gray')

In [ ]:
ATP = apartTimePrice("서울", "강서구", "엠밸리")
df_target,bdongCode =ATP.extractTargetPrice()

In [ ]:
df_sido

In [ ]:
# plt.figure(figsize = (12, 5))
# sns.scatterplot(df_target.loc[(df_target['isReal']==True)|(df_target['predict']==True),'contractDate'], df_target.loc[(df_target['isReal']==True)|(df_target['predict']==True),'averagePyeong'], hue = df_target['areaSixGroupNo'])

In [ ]:
plt.figure(figsize = (12, 5))
sns.scatterplot(df_target.loc[(df_target['isReal']==True)|(df_target['predict']==True),'contractDate'], df_target.loc[(df_target['isReal']==True)|(df_target['predict']==True),'averagePyeong'], hue = df_target['predict'])

In [ ]:
sql = f"""
SELECT *
FROM `aidepartners.aide.aide_apartment_price_origin`
WHERE sidoCode LIKE '11%'
      AND isReal = True
      AND yearMonth >='2018-03-01'
"""
df_sido = BH.read_table(sql)

In [ ]:
test = df_sido.groupby('yearMonth')['averagePyeong'].mean().reset_index()
test2 = df_sido[df_sido['sigunguCode']=='11500'].groupby('yearMonth')['averagePyeong'].mean().reset_index()
test3 = df_sido[df_sido['bdongCode']=='1150010500'].groupby('yearMonth')['averagePyeong'].mean().reset_index()
test4 = df_sido[df_sido['sigunguCode']=='11710'].groupby('yearMonth')['averagePyeong'].mean().reset_index()
test5 = df_sido[df_sido['bdongCode']=='1171010100'].groupby('yearMonth')['averagePyeong'].mean().reset_index()

In [ ]:
df_target[(df_target['isReal']==True)&(df_target['yearMonth']>='2020-01-01')].sort_values(by ='yearMonth')

In [ ]:
plt.figure(figsize = (14, 5))
sns.scatterplot(test['yearMonth'], test['averagePyeong'], label = '서울')
sns.scatterplot(test2['yearMonth'], test2['averagePyeong'], label = '강서구')
sns.scatterplot(test3['yearMonth'], test3['averagePyeong'], label = '마곡동')
sns.scatterplot(df_target.loc[((df_target['isReal']==True)|(df_target['predict']==True)|(df_target['predict']==True))&(df_target['yearMonth']>='2018-03-01')&(df_target['areaSixGroupNo']=='A3'),'yearMonth'], df_target.loc[((df_target['isReal']==True)|(df_target['predict']==True))&(df_target['yearMonth']>='2018-03-01')&(df_target['areaSixGroupNo']=='A3'),'averagePyeong'],label = '엠밸리')